In [1]:
! pip3 install datasets transformers

     |████████████████████████████████| 542 kB 7.3 MB/s 
     |████████████████████████████████| 2.6 MB 13.4 MB/s 
     |████████████████████████████████| 76 kB 4.9 MB/s 
     |████████████████████████████████| 43 kB 1.8 MB/s 
     |████████████████████████████████| 243 kB 56.3 MB/s 
     |████████████████████████████████| 118 kB 60.5 MB/s 
     |████████████████████████████████| 3.3 MB 54.7 MB/s 
     |████████████████████████████████| 895 kB 44.4 MB/s 
     |████████████████████████████████| 636 kB 59.3 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import numpy as np
import random
import torch
seed = 2001
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
# torch.cuda.manual_seed(seed) # if cuda
# torch.cuda.manual_seed_all(seed)
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark =False

In [3]:
import transformers

print(transformers.__version__)

4.9.0


In [4]:
#!pip install tqdm
import datasets
from datasets import Dataset, DatasetDict

In [5]:
import sys
import numpy as np
sys.path.append('/content/drive/MyDrive/nlp')
#np.save('/content/drive/MyDrive/nlp/test', np.array([2,3,4,5]))

If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/question-answering).

# Fine-tuning a model on a question-answering task

In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model to a question answering task, which is the task of extracting the answer to a question from a given context. We will see how to easily load a dataset for these kinds of tasks and use the `Trainer` API to fine-tune a model on it.

![Widget inference representing the QA task](https://github.com/huggingface/notebooks/blob/master/examples/images/question_answering.png?raw=1)

**Note:** This notebook finetunes models that answer question by taking a substring of a context, not by generating new text.

In [6]:
 
squad_v2 = True#False
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

Outline of the solution:
1. Converting the original data to a question answering format by adding questions such as 'who' + relation + entity B.
2. Adding questions with no answers in the training set and in validation set.
   For training - one extra random question for each sentence
   For validation - adding all the other questions (7 in total)
3. During test we check the answers given to all the questions related to a 
   specific sentence and hopefully the best answer will correspond with the
   correct relation as all other questions have no answer and will get lower
   score.

## Loading the dataset

In [7]:
import requests, zipfile, io

import pandas as pd

def download_data():
    url = "https://www.dropbox.com/s/izi2x4sjohpzoot/relation_extraction_dataset.zip?dl=1"
    r = requests.get(url)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall()

download_data()


df = pd.read_pickle("relation_extraction_dataset.pkl")
df.reset_index(drop=True, inplace=True)
df.head()

,end_idx,entities,entity_spans,match,original_article,sentence,start_idx,string_id
0,1024,"[Lilium, Baillie Gifford]","[[3, 9], [151, 166]]",raising $35,Happy Friday!\n\nWe sincerely hope you and you...,"3) Lilium, a German startup that’s making an a...",1013,invested_in
1,1762,"[Facebook ’s, Giphy]","[[92, 102], [148, 153]]",acquisition,Happy Friday!\n\nWe sincerely hope you and you...,"Meanwhile, the UK’s watchdog on Friday announc...",1751,acquired_by
2,2784,"[Global-e, Vitruvian Partners]","[[27, 35], [94, 112]]",raised $60,Happy Friday!\n\nWe sincerely hope you and you...,Israeli e-commerce startup Global-e has raised...,2774,invested_in
3,680,"[Joris Van Der Gucht, Silverfin]","[[0, 19], [35, 44]]",founder,Hg is a leading investor in tax and accounting...,"Joris Van Der Gucht, co-founder at Silverfin c...",673,founded_by
4,2070,"[Tim Vandecasteele, Silverfin]","[[0, 17], [71, 80]]",founder,Hg is a leading investor in tax and accounting...,"Tim Vandecasteele, co-founder added: ""We want ...",2063,founded_by


In [8]:
id2label = dict()
for idx, label in enumerate(df.string_id.value_counts().index):
  id2label[idx] = label
label2id = {v:k for k,v in id2label.items()}
label2id

{'CEO_of': 3,
 'acquired_by': 1,
 'founded_by': 0,
 'invested_in': 2,
 'owned_by': 6,
 'partners_with': 5,
 'subsidiary_of': 4}

In [9]:
questions=['who founded ','who acquired ','who invested in ','who is the CEO of ','who is subsidiary of ','who is partner with ','who owns ']
q2label=dict()
for idx, q in enumerate(questions):
  q2label[q] = id2label[idx]
q2label

{'who acquired ': 'acquired_by',
 'who founded ': 'founded_by',
 'who invested in ': 'invested_in',
 'who is partner with ': 'partners_with',
 'who is subsidiary of ': 'subsidiary_of',
 'who is the CEO of ': 'CEO_of',
 'who owns ': 'owned_by'}

In [10]:
 
#add questions and answers to db text and span
df['question'] = df['string_id'].transform(lambda x: questions[label2id[x]])#[typ.feature.names[i] for i in x]
df['question'] =df['question'] +df['entities'].transform(lambda x: x[1]+'?')

def add_answer(entity, span):
     
    return {'text':[entity[0]],'answer_start':[span[0][0]]}

#df['answer'] = df.(lambda x: add_answer(df.entities, df.entity_spans))# for i in range(df.entity_spans.count()))#, axis=1)
df['answers']= df[['entities', 'entity_spans']].apply(lambda x: add_answer(*x), axis=1)
df['id']=df.index.astype('str')
df.rename(columns={'sentence':'context'}, inplace=True)

In [11]:
import numpy as np

#split train validation
train_idx= np.random.choice(df.shape[0], size=int(0.8*df.shape[0]), replace=False)
val_idx=np.array([x for x in range(df.shape[0]) if not x in train_idx])
#train_idx.shape, val_idx.shape, df.shape
df_train=df.iloc[train_idx]
df_val=df.iloc[val_idx]
df_train.shape

(9624, 11)

#add questions with no answrs both for train and validation

In [12]:

import ast
 

 
df_train_ext=df_train.copy()
df_size=df.shape[0]
df_train_ext['question_id']=df_train_ext['id']
df_train_ext['string_id']=df_train['string_id']
def add_questionsWno_answers(group, withAnswer=False, size=0):
    row0 = group.iloc[0]
    row=pd.DataFrame(row0)#.transpose())#,index=line.name)
    row=row.transpose()
   
    rel=row0['string_id']
     
    possible_questions= [questions[i] for i in label2id.values() if i!=label2id[rel]]
    
    if withAnswer:
      #count=1
      idx=np.random.choice(len(possible_questions))
      possible_questions=[possible_questions[idx]]
      
  
    df3=pd.concat([row]*(len(possible_questions)+1))
 
    for i,q in enumerate(possible_questions) :
      #print(int(df3.iloc[i+1].loc['id'])+idx)
      df3.iloc[i+1].loc['question']=q + df3.iloc[i+1].loc['entities'][1]+ '?'
      #df3.loc[i+1,'question']=q + df3.iloc[i+1].loc['entities'][1]+ '?'
     
      df3.iloc[i+1].loc['answers'] = str({'text':[],'answer_start':[]}) #str()
      df3.iloc[i+1].loc['string_id']  = q2label[q]
      df3.iloc[i+1].loc['question_id']=str(i+1 + size + 8*int(df3.iloc[i+1].loc['id']))
    return df3

##
## Add questions with no answers for train dataset, for every question we add one more random question that has no answer 
df_train_ext=df_train_ext.groupby('id', group_keys=False).apply(add_questionsWno_answers, (True), (df_size))
df_train_ext['answers']= df_train_ext['answers'].transform(lambda x:ast.literal_eval(x) if isinstance(x, str) else x)
df_train_ext.shape[0]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


19248

In [13]:
df_train_ext.head(16)

,end_idx,entities,entity_spans,match,original_article,context,start_idx,string_id,question,answers,id,question_id
0,1024,"[Lilium, Baillie Gifford]","[[3, 9], [151, 166]]",raising $35,Happy Friday!\n\nWe sincerely hope you and you...,"3) Lilium, a German startup that’s making an a...",1013,invested_in,who invested in Baillie Gifford?,"{'text': ['Lilium'], 'answer_start': [3]}",0,0
0,1024,"[Lilium, Baillie Gifford]","[[3, 9], [151, 166]]",raising $35,Happy Friday!\n\nWe sincerely hope you and you...,"3) Lilium, a German startup that’s making an a...",1013,founded_by,who founded Baillie Gifford?,"{'text': [], 'answer_start': []}",0,12032
1,1762,"[Facebook ’s, Giphy]","[[92, 102], [148, 153]]",acquisition,Happy Friday!\n\nWe sincerely hope you and you...,"Meanwhile, the UK’s watchdog on Friday announc...",1751,acquired_by,who acquired Giphy?,"{'text': ['Facebook ’s'], 'answer_start': [92]}",1,1
1,1762,"[Facebook ’s, Giphy]","[[92, 102], [148, 153]]",acquisition,Happy Friday!\n\nWe sincerely hope you and you...,"Meanwhile, the UK’s watchdog on Friday announc...",1751,owned_by,who owns Giphy?,"{'text': [], 'answer_start': []}",1,12040
10,76,"[Collibra, CapitalG]","[[52, 60], [90, 98]]",raised $100,Belgium/US-based data governance technology co...,Belgium/US-based data governance technology co...,65,invested_in,who invested in CapitalG?,"{'text': ['Collibra'], 'answer_start': [52]}",10,10
10,76,"[Collibra, CapitalG]","[[52, 60], [90, 98]]",raised $100,Belgium/US-based data governance technology co...,Belgium/US-based data governance technology co...,65,founded_by,who founded CapitalG?,"{'text': [], 'answer_start': []}",10,12112
100,46,"[Randy, CloudBlue]","[[30, 35], [77, 86]]",founded,"Prior to co-founding Xometry, Randy co-founded...","Prior to co-founding Xometry, Randy co-founded...",39,founded_by,who founded CloudBlue?,"{'text': ['Randy'], 'answer_start': [30]}",100,100
100,46,"[Randy, CloudBlue]","[[30, 35], [77, 86]]",founded,"Prior to co-founding Xometry, Randy co-founded...","Prior to co-founding Xometry, Randy co-founded...",39,invested_in,who invested in CloudBlue?,"{'text': [], 'answer_start': []}",100,12832
1000,3256,"[Scopely, Barcelona Dublin DIGIT Game Studio]","[[381, 388], [464, 498]]",founded,Mobile game publisher Scopely has raised 200 m...,Viswanathan former NEA general partner Greycro...,3249,founded_by,who founded Barcelona Dublin DIGIT Game Studio?,"{'text': ['Scopely'], 'answer_start': [381]}",1000,1000
1000,3256,"[Scopely, Barcelona Dublin DIGIT Game Studio]","[[381, 388], [464, 498]]",founded,Mobile game publisher Scopely has raised 200 m...,Viswanathan former NEA general partner Greycro...,3249,subsidiary_of,who is subsidiary of Barcelona Dublin DIGIT Ga...,"{'text': [], 'answer_start': []}",1000,20032


Add questions to validation dataset so for each question we ask all the 7 questions. Thus the question which will have an answer will describe the correct relation

In [14]:


df_size=df.shape[0]
df_val_ext=df_val.copy()
df_val_ext['question_id']=df_val_ext['id']
df_val_ext=df_val_ext.groupby('id', group_keys=False).apply(add_questionsWno_answers, (False), (df_size))
df_val_ext['answers']= df_val_ext['answers'].transform(lambda x:ast.literal_eval(x) if isinstance(x, str) else x)
df_val_ext.head(15)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,end_idx,entities,entity_spans,match,original_article,context,start_idx,string_id,question,answers,id,question_id
10000,75,"[Chiliz, Binance Chain]","[[26, 32], [76, 89]]",partnership with,Sports blockchain venture Chiliz has announced...,Sports blockchain venture Chiliz has announced...,59,partners_with,who is partner with Binance Chain?,"{'text': ['Chiliz'], 'answer_start': [26]}",10000,10000
10000,75,"[Chiliz, Binance Chain]","[[26, 32], [76, 89]]",partnership with,Sports blockchain venture Chiliz has announced...,Sports blockchain venture Chiliz has announced...,59,founded_by,who founded Binance Chain?,"{'text': [], 'answer_start': []}",10000,92032
10000,75,"[Chiliz, Binance Chain]","[[26, 32], [76, 89]]",partnership with,Sports blockchain venture Chiliz has announced...,Sports blockchain venture Chiliz has announced...,59,acquired_by,who acquired Binance Chain?,"{'text': [], 'answer_start': []}",10000,92033
10000,75,"[Chiliz, Binance Chain]","[[26, 32], [76, 89]]",partnership with,Sports blockchain venture Chiliz has announced...,Sports blockchain venture Chiliz has announced...,59,invested_in,who invested in Binance Chain?,"{'text': [], 'answer_start': []}",10000,92034
10000,75,"[Chiliz, Binance Chain]","[[26, 32], [76, 89]]",partnership with,Sports blockchain venture Chiliz has announced...,Sports blockchain venture Chiliz has announced...,59,CEO_of,who is the CEO of Binance Chain?,"{'text': [], 'answer_start': []}",10000,92035
10000,75,"[Chiliz, Binance Chain]","[[26, 32], [76, 89]]",partnership with,Sports blockchain venture Chiliz has announced...,Sports blockchain venture Chiliz has announced...,59,subsidiary_of,who is subsidiary of Binance Chain?,"{'text': [], 'answer_start': []}",10000,92036
10000,75,"[Chiliz, Binance Chain]","[[26, 32], [76, 89]]",partnership with,Sports blockchain venture Chiliz has announced...,Sports blockchain venture Chiliz has announced...,59,owned_by,who owns Binance Chain?,"{'text': [], 'answer_start': []}",10000,92037
10003,1335,"[Alexandre Dreyfus, Socios com]","[[41, 58], [78, 88]]",founder,What is blockchain technology for OK at this p...,The core concept is relatively simple As Alexa...,1328,founded_by,who founded Socios com?,"{'text': ['Alexandre Dreyfus'], 'answer_start'...",10003,10003
10003,1335,"[Alexandre Dreyfus, Socios com]","[[41, 58], [78, 88]]",founder,What is blockchain technology for OK at this p...,The core concept is relatively simple As Alexa...,1328,acquired_by,who acquired Socios com?,"{'text': [], 'answer_start': []}",10003,92056
10003,1335,"[Alexandre Dreyfus, Socios com]","[[41, 58], [78, 88]]",founder,What is blockchain technology for OK at this p...,The core concept is relatively simple As Alexa...,1328,invested_in,who invested in Socios com?,"{'text': [], 'answer_start': []}",10003,92057


In [15]:
#df_val_ext[df_val_ext.id=='3']
df_val[:5]#.shape[0]

,end_idx,entities,entity_spans,match,original_article,context,start_idx,string_id,question,answers,id,question_id
2,2784,"[Global-e, Vitruvian Partners]","[[27, 35], [94, 112]]",raised $60,Happy Friday!\n\nWe sincerely hope you and you...,Israeli e-commerce startup Global-e has raised...,2774,invested_in,who invested in Vitruvian Partners?,"{'text': ['Global-e'], 'answer_start': [27]}",2,2
39,17937,"[Agosto, Google Cloud]","[[0, 6], [130, 142]]",founded,Above the Trend Line: your industry rumor cent...,"Agosto, founded in 2001, helps businesses enha...",17930,founded_by,who founded Google Cloud?,"{'text': ['Agosto'], 'answer_start': [0]}",39,39
40,18088,"[Agosto, Pythian]","[[4, 10], [37, 44]]",acquisition,Above the Trend Line: your industry rumor cent...,The Agosto acquisition will solidify Pythian's...,18077,acquired_by,who acquired Pythian?,"{'text': ['Agosto'], 'answer_start': [4]}",40,40
42,19372,"[Louis Tetu, Coveo]","[[190, 200], [220, 225]]",& CEO,Above the Trend Line: your industry rumor cent...,"""From a business perspective, COVID-19 is both...",19367,CEO_of,who is the CEO of Coveo?,"{'text': ['Louis Tetu'], 'answer_start': [190]}",42,42
44,21163,"[Felix Van de Maele, Collibra]","[[118, 136], [160, 168]]",CEO of,Above the Trend Line: your industry rumor cent...,"""The impact of the global pandemic highlights ...",21157,CEO_of,who is the CEO of Collibra?,"{'text': ['Felix Van de Maele'], 'answer_start...",44,44


In [16]:

 
from datasets import Dataset, DatasetDict
dataset_rel_train = Dataset.from_pandas(df_train_ext)
dataset_rel_val = Dataset.from_pandas(df_val_ext)
ds_rel_val_org = Dataset.from_pandas(df_val)

In [17]:
 
datasets=DatasetDict({'train':dataset_rel_train, 'val':dataset_rel_val})

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.  

In [19]:
from datasets import load_dataset, load_metric

The `datasets` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set.

In [20]:
datasets

DatasetDict({
    train: Dataset({
        features: ['end_idx', 'entities', 'entity_spans', 'match', 'original_article', 'context', 'start_idx', 'string_id', 'question', 'answers', 'id', 'question_id', '__index_level_0__'],
        num_rows: 19248
    })
    val: Dataset({
        features: ['end_idx', 'entities', 'entity_spans', 'match', 'original_article', 'context', 'start_idx', 'string_id', 'question', 'answers', 'id', 'question_id', '__index_level_0__'],
        num_rows: 16849
    })
})

We can see the training, validation and test sets all have a column for the context, the question and the answers to those questions.

To access an actual element, you need to select a split first, then give an index:

In [21]:
datasets["train"][0]

{'__index_level_0__': 0,
 'answers': {'answer_start': [3], 'text': ['Lilium']},
 'context': '3) Lilium, a German startup that’s making an all-electric vertical takeoff and landing passenger jet became a ‘unicorn’ after raising $35 million from Baillie Gifford, the largest investor in Tesla after its billionaire owner Elon Musk.',
 'end_idx': '1024',
 'entities': ['Lilium', 'Baillie Gifford'],
 'entity_spans': [[3, 9], [151, 166]],
 'id': '0',
 'match': 'raising $35',
 'original_article': 'Happy Friday!\n\nWe sincerely hope you and yours are keeping healthy and safe. Please take care of yourself and others.\n\nThis week, our research team tracked more than 70 tech funding deals worth over €2 billion (!), as well as 15 M&A transactions, rumours, and related news stories across Europe, including Russia, Israel, and Turkey.\n\nMeanwhile, here’s an overview of the 10 biggest European tech news items for this week (subscribe to our free newsletter to get this roundup in your inbox every Mond

In [22]:
type(datasets["train"])

datasets.arrow_dataset.Dataset

We can see the answers are indicated by their start position in the text (here at character 515) and their full text, which is a substring of the context as we mentioned above.

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset (automatically decoding the labels in passing).

In [23]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    print(df.dtypes)
    display(HTML(df.to_html()))

In [24]:
show_random_elements(datasets["train"])

end_idx              object
entities             object
entity_spans         object
match                object
original_article     object
context              object
start_idx            object
string_id            object
question             object
answers              object
id                   object
question_id          object
__index_level_0__     int64
dtype: object


## Preprocessing the training data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [25]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/442 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

The following assertion ensures that our tokenizer is a fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, and we will need some of the special features they have for our preprocessing.

In [26]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

You can check which type of models have a fast tokenizer available and which don't on the [big table of models](https://huggingface.co/transformers/index.html#bigtable).

You can directly call this tokenizer on two sentences (one for the answer, one for the context):

In [27]:
tokenizer("What is your name?", "My name is Sylvain.")

{'input_ids': [101, 2054, 2003, 2115, 2171, 1029, 102, 2026, 2171, 2003, 25353, 22144, 2378, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

Now one specific thing for the preprocessing in question answering is how to deal with very long documents. We usually truncate them in other tasks, when they are longer than the model maximum sentence length, but here, removing part of the the context might result in losing the answer we are looking for. To deal with this, we will allow one (long) example in our dataset to give several input features, each of length shorter than the maximum length of the model (or the one we set as a hyper-parameter). Also, just in case the answer lies at the point we split a long context, we allow some overlap between the features we generate controlled by the hyper-parameter `doc_stride`:

In [28]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

Let's find one long example in our dataset:

In [29]:
for i, example in enumerate(datasets["train"]):
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > 384:
        break
example = datasets["train"][i]

Token indices sequence length is longer than the specified maximum sequence length for this model (659 > 512). Running this sequence through the model will result in indexing errors


Without any truncation, we get the following length for the input IDs:

In [30]:
len(tokenizer(example["question"], example["context"])["input_ids"])

659

Now, if we just truncate, we will lose information (and possibly the answer to our question):

In [31]:
len(tokenizer(example["question"], example["context"], max_length=max_length, truncation="only_second")["input_ids"])

384

Note that we never want to truncate the question, only the context, else the `only_second` truncation picked. Now, our tokenizer can automatically return us a list of features capped by a certain maximum length, with the overlap we talked above, we just have to tell it with `return_overflowing_tokens=True` and by passing the stride:

In [32]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride
)

Now we don't have one list of `input_ids`, but several: 

In [33]:
[len(x) for x in tokenized_example["input_ids"]]

[384, 384, 163]

And if we decode them, we can see the overlap:

In [34]:
for x in tokenized_example["input_ids"][:2]:
    print(tokenizer.decode(x))

[CLS] who acquired amd? [SEP] intel corporation's dominance of the microprocessor market and its aggressive business practices ; the ability of third party manufacturers to manufacture amd's products on a timely basis in sufficient quantities and using competitive technologies ; expected manufacturing yields for amd's products ; amd's ability to introduce products on a timely basis with features and performance levels that provide value to its customers ; global economic uncertainty ; the loss of a significant customer ; amd's ability to generate revenue from its semi - custom soc products ; the impact of the covid - 19 pandemic on amd's business, financial condition and results of operations ; political, legal, economic risks and natural disasters ; the impact of government actions and regulations such as export administration regulations, tariffs and trade protection measures ; potential security vulnerabilities ; potential it outages, data loss, data breaches and cyber - attacks ; u

Now this will give us some work to properly treat the answers: we need to find in which of those features the answer actually is, and where exactly in that feature. The models we will use require the start and end positions of these answers in the tokens, so we will also need to to map parts of the original context to some tokens. Thankfully, the tokenizer we're using can help us with that by returning an `offset_mapping`:

In [35]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride
)
print(tokenized_example["offset_mapping"][0][:100])

[(0, 0), (0, 3), (4, 12), (13, 15), (15, 16), (16, 17), (0, 0), (0, 5), (6, 17), (17, 18), (18, 19), (20, 29), (30, 32), (33, 36), (37, 42), (42, 45), (45, 48), (48, 51), (52, 58), (59, 62), (63, 66), (67, 77), (78, 86), (87, 96), (96, 97), (98, 101), (102, 109), (110, 112), (113, 118), (119, 124), (125, 138), (139, 141), (142, 153), (154, 156), (156, 157), (157, 158), (158, 159), (160, 168), (169, 171), (172, 173), (174, 180), (181, 186), (187, 189), (190, 200), (201, 211), (212, 215), (216, 221), (222, 233), (234, 246), (246, 247), (248, 256), (257, 270), (271, 277), (278, 281), (282, 284), (284, 285), (285, 286), (286, 287), (288, 296), (296, 297), (298, 300), (300, 301), (301, 302), (302, 303), (304, 311), (312, 314), (315, 324), (325, 333), (334, 336), (337, 338), (339, 345), (346, 351), (352, 356), (357, 365), (366, 369), (370, 381), (382, 388), (389, 393), (394, 401), (402, 407), (408, 410), (411, 414), (415, 424), (424, 425), (426, 432), (433, 441), (442, 453), (453, 454), (455

This gives, for each index of our input IDS, the corresponding start and end character in the original text that gave our token. The very first token (`[CLS]`) has (0, 0) because it doesn't correspond to any part of the question/answer, then the second token is the same as the characters 0 to 3 of the question:

In [36]:
first_token_id = tokenized_example["input_ids"][0][1]
offsets = tokenized_example["offset_mapping"][0][1]
print(tokenizer.convert_ids_to_tokens([first_token_id])[0], example["question"][offsets[0]:offsets[1]])

who who


So we can use this mapping to find the position of the start and end tokens of our answer in a given feature. We just have to distinguish which parts of the offsets correspond to the question and which part correspond to the context, this is where the `sequence_ids` method of our `tokenized_example` can be useful:

In [37]:
sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)

[None, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

It returns `None` for the special tokens, then 0 or 1 depending on whether the corresponding token comes from the first sentence past (the question) or the second (the context). Now with all of this, we can find the first and last token of the answer in one of our input feature (or if the answer is not in this feature):

In [38]:
answers = example["answers"]
start_char = answers["answer_start"][0]
end_char = start_char + len(answers["text"][0])

# Start token index of the current span in the text.
token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1

# End token index of the current span in the text.
token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1

# Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
offsets = tokenized_example["offset_mapping"][0]
if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
    # Move the token_start_index and token_end_index to the two ends of the answer.
    # Note: we could go after the last offset if the answer is the last word (edge case).
    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    end_position = token_end_index + 1
    print(start_position, end_position)
else:
    print("The answer is not in this feature.")

230 231


And we can double check that it is indeed the theoretical answer:

In [39]:
print(tokenizer.decode(tokenized_example["input_ids"][0][start_position: end_position+1]))
print(answers["text"][0])

amd
AMD


For this notebook to work with any kind of models, we need to account for the special case where the model expects padding on the left (in which case we switch the order of the question and the context):

In [40]:
pad_on_right = tokenizer.padding_side == "right"

Now let's put everything together in one function we will apply to our training set. In the case of impossible answers (the answer is in another feature given by an example with a long context), we set the cls index for both the start and end position. We could also simply discard those examples from the training set if the flag `allow_impossible_answers` is `False`. Since the preprocessing is already complex enough as it is, we've kept is simple for this part.

In [41]:
def prepare_train_features(examples):
    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [42]:
print((datasets['train'][:69]))

{'end_idx': ['1024', '1024', '1762', '1762', '76', '76', '46', '46', '3256', '3256', '1175', '1175', '266', '266', '121', '121', '531', '531', '1180', '1180', '276', '276', '123', '123', '107', '107', '88', '88', '3238', '3238', '124', '124', '34', '34', '140', '140', '2652', '2652', '4156', '4156', '5735', '5735', '2242', '2242', '3334', '3334', '8817', '8817', '2562', '2562', '3245', '3245', '3659', '3659', '64', '64', '781', '781', '1530', '1530', '1948', '1948', '2415', '2415', '42', '42', '2647', '2647', '8075'], 'entities': [['Lilium', 'Baillie Gifford'], ['Lilium', 'Baillie Gifford'], ['Facebook ’s', 'Giphy'], ['Facebook ’s', 'Giphy'], ['Collibra', 'CapitalG'], ['Collibra', 'CapitalG'], ['Randy', 'CloudBlue'], ['Randy', 'CloudBlue'], ['Scopely', 'Barcelona Dublin DIGIT Game Studio'], ['Scopely', 'Barcelona Dublin DIGIT Game Studio'], ['Dreyfus', 'Changpeng Zhao'], ['Dreyfus', 'Changpeng Zhao'], ['Chiliz', 'Cointelegraph'], ['Chiliz', 'Cointelegraph'], ['Lolli', 'Safeway'], ['Lol

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [43]:
features = prepare_train_features(datasets['train'][:5])
features

{'input_ids': [[101, 2040, 11241, 1999, 15358, 8751, 29360, 1029, 102, 1017, 1007, 13451, 5007, 1010, 1037, 2446, 22752, 2008, 1521, 1055, 2437, 2019, 2035, 1011, 3751, 7471, 19744, 1998, 4899, 4628, 6892, 2150, 1037, 1520, 21830, 1521, 2044, 6274, 1002, 3486, 2454, 2013, 15358, 8751, 29360, 1010, 1996, 2922, 14316, 1999, 26060, 2044, 2049, 22301, 3954, 3449, 2239, 14163, 6711, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command. Since our preprocessing changes the number of samples, we need to remove the old columns when applying it.

In [44]:
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

  0%|          | 0/20 [00:00<?, ?ba/s]

  0%|          | 0/17 [00:00<?, ?ba/s]

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready for training, we can download the pretrained model and fine-tune it. Since our task is question answering, we use the `AutoModelForQuestionAnswering` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us:

In [45]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this mode

In [46]:
#in case we already trained the model and want to continue
#model = AutoModelForQuestionAnswering.from_pretrained("/content/drive/MyDrive/nlp/test-rel-trained")

The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a `Trainer`, we will need to define three more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [47]:
args = TrainingArguments(
    f"test-rel",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay.

Then we will need a data collator that will batch our processed examples together, here the default one will work:

In [48]:
from transformers import default_data_collator

data_collator = default_data_collator

We will evaluate our model and compute metrics in the next section (this is a very long operation, so we will only compute the evaluation loss during training).

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [49]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["val"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

We can now finetune our model by just calling the `train` method:

In [50]:
trainer.train()
trainer.save_model("/content/drive/MyDrive/nlp/test-rel-trained")

Since this training is particularly long, let's save the model just in case we need to restart.

## Evaluation

Evaluating our model will require a bit more work, as we will need to map the predictions of our model back to parts of the context. The model itself predicts logits for the start and en position of our answers: if we take a batch from our validation datalaoder, here is the output our model gives us:

In [51]:
import torch

for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()

odict_keys(['loss', 'start_logits', 'end_logits'])

The output of the model is a dict-like object that contains the loss (since we provided labels), the start and end logits. We won't need the loss for our predictions, let's have a look a the logits:

In [52]:
output.start_logits.shape, output.end_logits.shape

(torch.Size([16, 384]), torch.Size([16, 384]))

We have one logit for each feature and each token. The most obvious thing to predict an answer for each featyre is to take the index for the maximum of the start logits as a start position and the index of the maximum of the end logits as an end position.

In [53]:
output.start_logits.argmax(dim=-1), output.end_logits.argmax(dim=-1)

(tensor([15,  0,  0,  0,  0,  0,  0, 15,  0,  0,  0,  0,  0,  0, 29,  0],
        device='cuda:0'),
 tensor([16,  0,  0,  0,  0,  0,  0, 18,  0,  0,  0,  0,  0,  0, 33,  0],
        device='cuda:0'))

This will work great in a lot of cases, but what if this prediction gives us something impossible: the start position could be greater than the end position, or point to a span of text in the question instead of the answer. In that case, we might want to look at the second best prediction to see if it gives a possible answer and select that instead.

However, picking the second best answer is not as easy as picking the best one: is it the second best index in the start logits with the best index in the end logits? Or the best index in the start logits with the second best index in the end logits? And if that second best answer is not possible either, it gets even trickier for the third best answer.


To classify our answers, we will use the score obtained by adding the start and end logits. We won't try to order all the possible answers and limit ourselves to with a hyper-parameter we call `n_best_size`. We'll pick the best indices in the start and end logits and gather all the answers this predicts. After checking if each one is valid, we will sort them by their score and keep the best one. Here is how we would do this on the first feature in the batch:

In [54]:
n_best_size = 20

In [55]:
import numpy as np

start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": "" # We need to find a way to get back the original substring corresponding to the answer in the context
                }
            )

And then we can sort the `valid_answers` according to their `score` and only keep the best one. The only point left is how to check a given span is inside the context (and not the question) and how to get back the text inside. To do this, we need to add two things to our validation features:
- the ID of the example that generated the feature (since each example can generate several features, as seen before);
- the offset mapping that will give us a map from token indices to character positions in the context.

That's why we will re-process the validation set with the following function, slightly different from `prepare_train_features`:

In [56]:
def prepare_validation_features(examples):
    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples['question_id'][sample_index]) #####__index_level_0__

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

And like before, we can apply that function to our validation set easily:

In [57]:
validation_features = datasets["val"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=datasets["val"].column_names
)

  0%|          | 0/17 [00:00<?, ?ba/s]

Now we can grab the predictions for all features by using the `Trainer.predict` method:

In [58]:
raw_predictions = trainer.predict(validation_features)

The following columns in the test set  don't have a corresponding argument in `DistilBertForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 16856
  Batch size = 16


The `Trainer` *hides* the columns that are not used by the model (here `example_id` and `offset_mapping` which we will need for our post-processing), so we set them back:

In [59]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

We can now refine the test we had before: since we set `None` in the offset mappings when it corresponds to a part of the question, it's easy to check if an answer is fully inside the context. We also eliminate very long answers from our considerations (with an hyper-parameter we can tune)

In [60]:
max_answer_length = 30

In [61]:
start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
offset_mapping = validation_features[0]["offset_mapping"]
# The first feature comes from the first example. For the more general case, we will need to be match the example_id to
# an example index
context = datasets["val"][0]["context"]
print(context)
# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
        # to part of the input_ids that are not in the context.
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        # Don't consider answers with a length that is either < 0 or > max_answer_length.
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
valid_answers

Sports blockchain venture Chiliz has announced a strategic partnership with Binance Chain the mainnet of major cryptocurrency exchange


[{'score': 16.619637, 'text': 'Chiliz'},
 {'score': 10.727211, 'text': 'Chiliz has'},
 {'score': 8.035167, 'text': 'Chili'},
 {'score': 7.923586, 'text': 'Chiliz has announced'},
 {'score': 6.628046, 'text': 'Chiliz has announced a'},
 {'score': 6.403385, 'text': 'z'},
 {'score': 5.088732, 'text': 'Sports blockchain venture Chiliz'},
 {'score': 4.298773, 'text': 'blockchain venture Chiliz'},
 {'score': 3.1134443, 'text': 'Chiliz has announced a strategic partnership'},
 {'score': 2.943778, 'text': 'Chiliz has announced a strategic'},
 {'score': 2.8883576, 'text': 'venture Chiliz'},
 {'score': 2.600844, 'text': 'chain venture Chiliz'},
 {'score': 2.1659794, 'text': 'n venture Chiliz'},
 {'score': 1.6354704,
  'text': 'Chiliz has announced a strategic partnership with Binance Chain the mainnet'},
 {'score': 1.621232,
  'text': 'Chiliz has announced a strategic partnership with Binance Chain the mainnet of major cryptocurren'},
 {'score': 1.5599504,
  'text': 'Chiliz has announced a strat

We can compare to the actual ground-truth answer:

In [62]:
datasets["val"][0]["answers"]

{'answer_start': [26], 'text': ['Chiliz']}

Our model picked the right as the most likely answer!

As we mentioned in the code above, this was easy on the first feature because we knew it comes from the first example. For the other features, we will need a map between examples and their corresponding features. Also, since one example can give several features, we will need to gather together all the answers in all the features generated by a given example, then pick the best one. The following code builds a map from example index to its corresponding features indices:

In [63]:
import collections

examples = datasets["val"]
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["question_id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

We're almost ready for our post-processing function. The last bit to deal with is the impossible answer (when `squad_v2 = True`). The code above only keeps answers that are inside the context, we need to also grab the score for the impossible answer (which has start and end indices corresponding to the index of the CLS token). When one example gives several features, we have to predict the impossible answer when all the features give a high score to the impossible answer (since one feature could predict the impossible answer just because the answer isn't in the part of the context it has access too), which is why the score of the impossible answer for one example is the *minimum* of the scores for the impossible answer in each feature generated by the example.

We then predict the impossible answer when that score is greater than the score of the best non-impossible answer. All combined together, this gives us this post-processing function:

And we can apply our post-processing function to our raw predictions - This is the original versiom which prepare every question seperately:

In [64]:
from tqdm.auto import tqdm

def postprocess_qa_predictionsORG(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["question_id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]
         
        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                 
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        
        if not squad_v2:
            predictions[example["question_id"]] = best_answer["text"]
        else:
             
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["question_id"]] = answer

    return predictions

In [65]:
final_predictions_org = postprocess_qa_predictionsORG(datasets["val"], validation_features, raw_predictions.predictions)
len(final_predictions_org)

Post-processing 16849 example predictions split into 16856 features.


  0%|          | 0/16849 [00:00<?, ?it/s]

16849

In this version we preprocess together all the question related to a specific sentence ('id') - we aggregate all the answers for these questions and order them and hopefuly the best answer will correspond with the correct relation as all other questions have no answer and will get lower score.

In [66]:
from tqdm.auto import tqdm
RELֹֹ_NUM=7
def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30, rel_num=7):
    print(rel_num)
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["question_id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]
        #print(example, feature_indices)
        min_null_score = None # Only used if squad_v2 is True.
        if example_index%rel_num ==0 :
          valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
       # print('fff', len(feature_indices))
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                #print("jj", feature_null_score)
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": example['string_id']##context[start_char: end_char]
                        }
                    )
        if (example_index%rel_num ==(rel_num-1)):
            if (len(valid_answers) ) > 0:
                best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
            else:
                # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
                # failure.
                best_answer = {"text": "", "score": 0.0}
            
            # Let's pick our final answer: the best one or the null answer (only for squad_v2)
            
            if not squad_v2:
                predictions[example["id"]] = best_answer["text"]
            else:
                
                answer = best_answer["text"] ###if best_answer["score"] > min_null_score else ""
                predictions[example["id"]] = answer

    return predictions

In [67]:

final_predictions = postprocess_qa_predictions(datasets["val"], validation_features, raw_predictions.predictions)

7
Post-processing 16849 example predictions split into 16856 features.


  0%|          | 0/16849 [00:00<?, ?it/s]

In [68]:
len(final_predictions)

2407

Then we can load the metric from the datasets library.

In [69]:
metric = load_metric("squad_v2" if squad_v2 else "squad")

Downloading:   0%|          | 0.00/2.26k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.18k [00:00<?, ?B/s]

Then we can call compute on it. We just need to format predictions and labels a bit as it expects a list of dictionaries and not one big dictionary. In the case of squad_v2, we also have to set a `no_answer_probability` argument (which we set to 0.0 here as we have already set the answer to empty if we picked it).

In [70]:
# how good is the model in the regular question answering task
if squad_v2:
    formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions_org.items()]
else:
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions_org.items()]
references = [{"id": ex["question_id"], "answers": ex["answers"]} for ex in datasets["val"]]
metric.compute(predictions=formatted_predictions, references=references)

{'HasAns_exact': 88.20108018280017,
 'HasAns_f1': 91.13335286787724,
 'HasAns_total': 2407,
 'NoAns_exact': 98.57360476388313,
 'NoAns_f1': 98.57360476388313,
 'NoAns_total': 14442,
 'best_exact': 97.09181553801413,
 'best_exact_thresh': 0.0,
 'best_f1': 97.51071163588203,
 'best_f1_thresh': 0.0,
 'exact': 97.09181553801413,
 'f1': 97.51071163588216,
 'total': 16849}

In [73]:
print(formatted_predictions[:9])
references[:9]

[{'id': '10000', 'prediction_text': 'partners_with', 'no_answer_probability': 0.0}, {'id': '10003', 'prediction_text': 'founded_by', 'no_answer_probability': 0.0}, {'id': '10008', 'prediction_text': 'CEO_of', 'no_answer_probability': 0.0}, {'id': '1002', 'prediction_text': 'acquired_by', 'no_answer_probability': 0.0}, {'id': '10023', 'prediction_text': 'founded_by', 'no_answer_probability': 0.0}, {'id': '10030', 'prediction_text': 'CEO_of', 'no_answer_probability': 0.0}, {'id': '10042', 'prediction_text': 'subsidiary_of', 'no_answer_probability': 0.0}, {'id': '10044', 'prediction_text': 'invested_in', 'no_answer_probability': 0.0}, {'id': '10046', 'prediction_text': 'CEO_of', 'no_answer_probability': 0.0}]


[{'answers': 'invested_in', 'id': '2'},
 {'answers': 'founded_by', 'id': '39'},
 {'answers': 'acquired_by', 'id': '40'},
 {'answers': 'CEO_of', 'id': '42'},
 {'answers': 'CEO_of', 'id': '44'},
 {'answers': 'acquired_by', 'id': '51'},
 {'answers': 'partners_with', 'id': '53'},
 {'answers': 'acquired_by', 'id': '60'},
 {'answers': 'partners_with', 'id': '61'}]

We now test it on our original task of relation detection

In [71]:
if squad_v2:
    formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
else:
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["string_id"]} for ex in ds_rel_val_org]
y_true=[ref['answers'] for ref in references]
y_true[:5]
#metric.compute(predictions=formatted_predictions, references=references)

['invested_in', 'founded_by', 'acquired_by', 'CEO_of', 'CEO_of']

In [72]:
#len(final_predictions)
my_predictions=dict(sorted(final_predictions.items(), key=lambda x:int(x[0])))
#list(my_predictions.items())[:5]
y_pred=list(my_predictions.values())
y_pred[:9]

['invested_in',
 'founded_by',
 'acquired_by',
 'CEO_of',
 'founded_by',
 'acquired_by',
 'partners_with',
 'acquired_by',
 'partners_with']

In [83]:
from sklearn.metrics import f1_score, classification_report
#f1_score(y_true,y_pred, average=None)#'macro')
print(classification_report(y_true, y_pred))

               precision    recall  f1-score   support

       CEO_of       0.96      0.90      0.93       341
  acquired_by       0.99      0.98      0.99       494
   founded_by       0.96      0.99      0.97       751
  invested_in       0.98      0.98      0.98       459
     owned_by       1.00      0.99      0.99        97
partners_with       0.97      0.99      0.98       115
subsidiary_of       0.98      0.97      0.98       150

     accuracy                           0.97      2407
    macro avg       0.98      0.97      0.98      2407
 weighted avg       0.97      0.97      0.97      2407

